### Constants

In [ ]:
import random
import string

PROJECT_ID = "jchavezar-demo"
REGION = "us-central1"  # @param {type: "string"}
UUID = "".join(random.choices(string.ascii_lowercase + string.digits, k=8))
BUCKET_NAME = "vtx-demos"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"
DATA_PATH = "data"
# Base
DATASET_NAME = "news_corpora"
DATASET_URI = f"{BUCKET_URI}/{DATA_PATH}/raw/newsCorpora.csv"

# Experiments
TASK = "classification"
MODEL_TYPE = "naivebayes"
EXPERIMENT_NAME = f"{TASK}-{MODEL_TYPE}-{UUID}"
EXPERIMENT_RUN_NAME = "run-1"

# Preprocessing
PREPROCESSED_DATASET_NAME = f"preprocessed_{DATASET_NAME}"
PREPROCESS_EXECUTION_NAME = "preprocess"
COLUMN_NAMES = [
    "id",
    "title",
    "url",
    "publisher",
    "category",
    "story",
    "hostname",
    "timestamp",
]
DELIMITER = "	"
INDEX_COL = 0
PREPROCESSED_DATASET_URI = (
    f"{BUCKET_URI}/{DATA_PATH}/preprocess/{PREPROCESSED_DATASET_NAME}.csv"
)

# Training
TRAIN_EXECUTION_NAME = "train"
TARGET = "category"
FEATURES = "title"
TEST_SIZE = 0.2
SEED = 8
TRAINED_MODEL_URI = f"{BUCKET_URI}/deliverables/{UUID}"
MODEL_NAME = f"{EXPERIMENT_NAME}-model"
SERVE_IMAGE = "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"

### Get Data 

In [ ]:
!mkdir -m 777 -p {DATA_PATH}

DATASET_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip"
!wget --no-parent {DATASET_URL} --directory-prefix={DATA_PATH}
!mkdir -m 777 -p {DATA_PATH}/temp {DATA_PATH}/raw
!unzip {DATA_PATH}/*.zip -d {DATA_PATH}/temp
!mv {DATA_PATH}/temp/*.csv {DATA_PATH}/raw && rm -Rf {DATA_PATH}/temp && rm -f {DATA_PATH}/*.zip
!gsutil -m cp -R {DATA_PATH}/raw $BUCKET_URI/{DATA_PATH}/raw

--2022-09-27 15:35:40--  https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 29224203 (28M) [application/x-httpd-php]
Saving to: ‘data/NewsAggregatorDataset.zip’

NewsAggregatorDatas 100%[===================>]  27.87M  34.2MB/s    in 0.8s    

2022-09-27 15:35:41 (34.2 MB/s) - ‘data/NewsAggregatorDataset.zip’ saved [29224203/29224203]

Archive:  data/NewsAggregatorDataset.zip
  inflating: data/temp/2pageSessions.csv  
   creating: data/temp/__MACOSX/
  inflating: data/temp/__MACOSX/._2pageSessions.csv  
  inflating: data/temp/newsCorpora.csv  
  inflating: data/temp/__MACOSX/._newsCorpora.csv  
  inflating: data/temp/readme.txt    
  inflating: data/temp/__MACOSX/._readme.txt  
Copying file://data/raw/newsCorpora.csv [Content-Type=text/c

### Import Libraries

In [ ]:
# General
import logging
import collections
import tempfile
import time
from json import dumps

collections.Iterable = collections.abc.Iterable

# Vertex AI
from google.cloud import aiplatform as vertex_ai

### Initialize Vertex AI


In [ ]:
vertex_ai.init(
    project=PROJECT_ID, experiment=EXPERIMENT_NAME, staging_bucket=BUCKET_URI
)

### Initialize Experiment Run

In [ ]:
run = vertex_ai.start_run(EXPERIMENT_RUN_NAME)

Associating projects/569083142710/locations/us-central1/metadataStores/default/contexts/classification-naivebayes-kkafjjcq-run-1 to Experiment: classification-naivebayes-kkafjjcq
